# Compare only 1 Body's Simfile

In [2]:
import sys,os

import numpy as np
import matplotlib as mpl
import matplotlib.cm as cm
import matplotlib.pyplot as plt
import matplotlib.colors as colors
import matplotlib.gridspec as gridspec


#this works apparently only for savefig stuff
mpl.rcParams['figure.figsize']=(16.0,8.0)    #(6.0,4.0)
mpl.rcParams['font.size']=12                #10 
mpl.rcParams['savefig.dpi']=100             #72 

mpl.rcParams['figure.subplot.bottom']=.1    #.125


plt.rc('font', family='serif')
plt.rc('text', usetex=True)

#inline Shit
%matplotlib inline
%config InlineBackend.figure_format='svg'
%config InlineBackend.rc = {'figure.facecolor': 'white', 'figure.subplot.bottom': 0.125, 'figure.edgecolor': 'white', 'savefig.dpi': 300, 'figure.figsize': (12.0, 8.0), 'font.size': 10}

#GUi shit
%matplotlib tk

mpl.get_configdir()

# Import General Packages from me
from Tools.Parsers import *
from Tools.BoundingBox import *


In [8]:
def plotBodies():
        
    plt.close('all')
    
    nBodies = 1
    
    width_ratios=[1]
    height_ratios=[3,3]
    nSubplots  = 2      
                           
    if(dynState2.bodies[bodyIdx].additionalBytesType == 5):
        height_ratios.append(1)
        nSubplots +=1
        
    if(calculateDiff):
        height_ratios.append(1)
        height_ratios.append(1)
        nSubplots +=2
        
    if(calculateTol):
        height_ratios.append(3)
        nSubplots +=1    
    
    gs = gridspec.GridSpec(nSubplots, 1, width_ratios=width_ratios, height_ratios=height_ratios)
    
    jet = plt.get_cmap('jet') 
    cNorm  = colors.Normalize(vmin=0, vmax=nBodies)
    scalarMap = cm.ScalarMappable(norm=cNorm, cmap=jet)
    
    fig1 = plt.figure()
    ax10 = plt.subplot(gs[0]);
    # Position       
    ax10.plot(dynState1.t, dynState1.bodies[bodyIdx].q[:,0:3], color= scalarMap.to_rgba(0), label=label1, marker='.');
    ax10.plot(dynState2.t, dynState2.bodies[bodyIdx].q[:,0:3], color= scalarMap.to_rgba(0), linestyle='-.', label=label2, marker='.');
    
    ax11 = plt.subplot(gs[1], sharex= ax10);
    ax11.plot(dynState1.t, dynState1.bodies[bodyIdx].q[:,3:7], color= scalarMap.to_rgba(0),marker='.');
    ax11.plot(dynState2.t, dynState2.bodies[bodyIdx].q[:,3:7], color= scalarMap.to_rgba(0),linestyle='-.',marker='.');
    
    gsIdx = 1;
    if(dynState1.bodies[bodyIdx].additionalBytesType == 5):
        gsIdx+=1
        ax12 = plt.subplot(gs[gsIdx], sharex= ax10);
        ax12.plot( dynState1.t , dynState1.bodies[bodyIdx].addData["rank"] , color= scalarMap.to_rgba(0), marker='o')
        ax12.grid(True)
        ax12.set_ymargin(0.2)
        ax12.set_title('MPI Ranks %s' % label1 );
        ax12.autoscale(True,'both',False)
        
    if(calculateTol):
        gsIdx+=1
        ax14 = plt.subplot(gs[gsIdx], sharex= ax10);
        ax14.plot( dynState1.t , absDiffQMax , color= 'red', linestyle='-')
        ax14.grid(True)
        ax14.set_ymargin(0.2)
        ax14.set_title('Max. Abs Tol');
        ax14.autoscale(True,'both',False)
        
    if(calculateDiff):
        # q
        gsIdx+=1
        ax13 = plt.subplot(gs[gsIdx], sharex = ax10);    
        # make a color map of fixed colors
        cmap = mpl.colors.ListedColormap(['red','green'])
        bounds=[0,0.5,1]
        norm = mpl.colors.BoundaryNorm(bounds, cmap.N)
        # tell imshow about color map so that only set colors are used
        im = plt.imshow(difference[0],interpolation='nearest', cmap = cmap, norm=norm, extent = [dynState1.t[0],dynState1.t[minRange-1],0,1])
        ax13.plot(dynState1.t[0:minRange],difference[0][0,:],'ko')
        plt.gca().set_aspect('auto')
        
        # time
        gsIdx+=1
        ax14 = plt.subplot(gs[gsIdx], sharex = ax10);    
        # make a color map of fixed colors
        cmap = mpl.colors.ListedColormap(['red','green'])
        bounds=[0,0.5,1]
        norm = mpl.colors.BoundaryNorm(bounds, cmap.N)
        # tell imshow about color map so that only set colors are used
        im = ax14.imshow(difference[2],interpolation='nearest', cmap = cmap, norm=norm, extent = [dynState1.t[0],dynState1.t[minRange-1],0,1])
        ax14.plot(dynState1.t[0:minRange],difference[2][0,:],'ko')
        ax13.set_title('Validity q');
        ax14.set_title('Validity t');
    
    
    
        
    ax10.legend()
    ax10.grid(True)
    ax10.set_title('Position, Timesteps: ' + str(timesteps));
    ax10.set_ylabel('\mathbf{r}_S(t)');
    ax10.set_ymargin(0.2)
    ax10.autoscale(True,'both',False)
    
    ax11.grid(True)
    ax11.set_title('Quaternion');
    ax11.set_ylabel('\mathbf{a}_{KI}(t)');
    ax11.set_ymargin(0.2)
    ax11.autoscale(True,'both',False)
    
    
    
    gs.tight_layout(fig1)
    plt.show()
    
    
    # Velocity
    fig2 = plt.figure()
    ax20 = plt.subplot(gs[0]);           
    ax20.plot(dynState1.t, dynState1.bodies[bodyIdx].u[:,0:3], color= scalarMap.to_rgba(0),marker='.');
    ax20.plot(dynState2.t, dynState2.bodies[bodyIdx].u[:,0:3], color= scalarMap.to_rgba(0),linestyle='-.',marker='.');
    
    ax21 = plt.subplot(gs[1], sharex=ax20);
    ax21.plot(dynState1.t, dynState1.bodies[bodyIdx].u[:,3:6], color= scalarMap.to_rgba(0),marker='.');
    ax21.plot(dynState2.t, dynState2.bodies[bodyIdx].u[:,3:6], color= scalarMap.to_rgba(0),linestyle='-.',marker='.');
    
    gsIdx = 1;
    
    if(dynState2.bodies[bodyIdx].additionalBytesType == 5):
        gsIdx+=1
        ax22 = plt.subplot(gs[gsIdx], sharex= ax20);
        ax22.plot( dynState2.t , dynState2.bodies[bodyIdx].addData["rank"] , color= scalarMap.to_rgba(0), marker='o')
        ax22.grid(True)
        ax22.set_ymargin(0.2)
        ax22.set_title('MPI Ranks %s' % label2);
        ax22.autoscale(True,'both',False)
        
    if(calculateTol):
        gsIdx+=1
        ax24 = plt.subplot(gs[gsIdx], sharex= ax20);
        ax24.plot( dynState1.t , absDiffUMax , color= 'blue', linestyle='-')
        ax24.grid(True)
        ax24.set_ymargin(0.2)
        ax24.set_title('Max. Abs Tol');
        ax24.autoscale(True,'both',False)
        
    if(calculateDiff):
        gsIdx+=1
        ax23 = plt.subplot(gs[gsIdx], sharex = ax20);    
        # make a color map of fixed colors
        cmap = mpl.colors.ListedColormap(['red','green'])
        bounds=[0,0.5,1]
        norm = mpl.colors.BoundaryNorm(bounds, cmap.N)
        # tell imshow about color map so that only set colors are used
        im = ax23.imshow(difference[1],interpolation='nearest', cmap = cmap, norm=norm, extent = [dynState1.t[0],dynState1.t[minRange-1],0,1])
        plt.gca().set_aspect('auto')
        ax23.plot(dynState1.t[0:minRange],difference[1][0,:],'ko')
        ax23.set_title('Validity u');
    
    
    ax20.grid(True)
    ax20.set_title('Velocity');
    ax20.set_ylabel('\mathbf{v}_S(t)');
    ax20.set_xmargin(0.0)
    ax20.set_ymargin(0.1)
    
    ax21.grid(True)
    ax21.set_title('Angular Velocity');
    ax21.set_ylabel('\mathbf{\omega}_{KI}(t)');
    ax21.set_xmargin(0.0)
    ax21.set_ymargin(0.1)
    
    
    
    gs.tight_layout(fig2)
    plt.show()

In [10]:
scriptpath = "./"
# Add the directory containing your module to the Python path (wants absolute paths)
sys.path.append(os.path.abspath(scriptpath))

SimFileReader.ReadInSimFile

bodyIdx = 456
bodyRange = [bodyIdx,bodyIdx]
label1='Grid';
label2='KdTree';


timeRange = [0,1.5]
calculateDiff = True
calculateTol = True
readLogs = False; # Not implemented so far
folderNumber = 0
#For single file which contains multiple bodies
dynState1 = SimFileReader.ReadInSimFile("./../SimFiles/SimDataRECORDGRID/SimState.sim" ,timeRange,bodyRange,False)
print(dynState1)
#print(dynState1.bodies)

folderNumber2 = 1
dynState2 = SimFileReader.ReadInSimFile("./../SimFiles/SimDataRECORDKDTREE/SimState.sim" ,timeRange,bodyRange,False)
# dynState2 = ReadInSimFile("./../GlobalFolder/SimFiles/SimDataRECORDMPI_%d/SimState.sim" % folderNumber2, timeRange , bodyRange , False)
print(dynState2)
#print(dynState2.bodies)

if(len(dynState1.t) != len(dynState2.t)):
    print("WARNING: STATES have no equal length!")

timesteps = len(dynState1.t)


if(calculateDiff):

    minRange = min(len(dynState1.t),len(dynState2.t))
    difference = ( np.zeros((1,minRange)) , np.zeros((1,minRange)), np.zeros((1,minRange)));  #(differences in q, diffs in u, t)
    for i in range(minRange):
        if( (dynState1.bodies[bodyIdx][0][i,0:7] == dynState2.bodies[bodyIdx][0][i,0:7]).all() ):
            difference[0][0,i] = 1;
            
        if( (dynState1.bodies[bodyIdx][1][i,0:6] == dynState2.bodies[bodyIdx][1][i,0:6]).all() ):
            difference[1][0,i] = 1;
            
        if( dynState1.t[i] == dynState2.t[i]):
            difference[2][0,i] = 1;
            
if(calculateTol):
    absDiffU = np.absolute(dynState1.bodies[bodyIdx][1]-dynState2.bodies[bodyIdx][1])
    absDiffUMax = absDiffU.max(axis=1)
    absDiffQ = np.absolute(dynState1.bodies[bodyIdx][0]-dynState2.bodies[bodyIdx][0])
    absDiffQMax = absDiffQ.max(axis=1)
    
plotBodies();

Read in SimFile at: ./../SimFiles/SimDataRECORDGRID/SimState.sim
--> File has: 1134.2308387756348 mb
--> SimFile signature: b'MBSF' found
--> SimFile version: 2
--> nSimBodies: 10000
--> nDofqObj: 7
--> nDofuObj: 6
--> Add. Bytes Type: 5
--> Add. Bytes Per Body: 20
--> nBytesHeader: 28
--> nBytesBodyState: 132
--> nBytesState: 1320008
--> nStates: 901
--> nAdditionalWrongBytes: 0
--> minTime: 0.0
--> maxTime: 1.7999999999998344
Filtering Simfile: t = [0, 1.5]
--> readNStates: 752
--> Time t: [ 0.     0.002  0.004  0.006  0.008  0.01   0.012  0.014  0.016  0.018
  0.02   0.022  0.024  0.026  0.028  0.03   0.032  0.034  0.036  0.038
  0.04   0.042  0.044  0.046  0.048  0.05   0.052  0.054  0.056  0.058
  0.06   0.062  0.064  0.066  0.068  0.07   0.072  0.074  0.076  0.078
  0.08   0.082  0.084  0.086  0.088  0.09   0.092  0.094  0.096  0.098  0.1
  0.102  0.104  0.106  0.108  0.11   0.112  0.114  0.116  0.118  0.12
  0.122  0.124  0.126  0.128  0.13   0.132  0.134  0.136  0.138  0.14
  0

In [56]:
dynState1.bodies[741].addData.rank

AttributeError: 'dict' object has no attribute 'rank'